In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
import json

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Argus/17-02-2015/timeout1/UNSW-NB15_Argus_1.csv'
df = pd.read_csv(out_dir)

In [4]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/ET/ET_unsw_argus_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)

In [5]:
# Features used in paper 2 modified names
cols = ["Dur", "RunTime", "IdleTime", "Mean", "StdDev", "Sum", "Min", "Max", "Proto", "Cause", "TotPkts", "SrcPkts", "DstPkts", "TotBytes", "SrcBytes",
         "DstBytes", "Load", "SrcLoad", "DstLoad", "Rate", "SrcRate", "DstRate", "Attack",
        "SAppBytes", "DAppBytes", "SynAck", "AckDat", "TcpRtt"
        ]

len(cols)

28

In [6]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]

### Training

In [7]:
best_f1 = 0
worst_f1 = 1


best_mean, worst_mean, best_std, worst_std = None, None, None, None
save=True

for timeout in timeouts:
    print("Processing timeout : ", timeout)    
    if timeout =='default':
        out_dir1 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Argus/17-02-2015/{timeout}/UNSW-NB15_Argus_{timeout}.csv'
        out_dir2 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Argus/22-01-2015//{timeout}/UNSW-NB15_Argus_{timeout}.csv'
    else:
        out_dir1 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Argus/17-02-2015/timeout{timeout}/UNSW-NB15_Argus_{timeout}.csv'
        out_dir2 = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/unsw/Argus/22-01-2015/timeout{timeout}/UNSW-NB15_Argus_{timeout}.csv'

    df1 = pd.read_csv(out_dir1)
    df2 = pd.read_csv(out_dir2)
        
    df = df1.append(df2, ignore_index=True)
    df = df[cols]

    
    #df = df.drop(columns=['SrcId', 'SrcAddr', 'DstAddr', 'Sport', 'Dport', 'StartTime', 'LastTime', 'Dir', 'State', 'sIpId', 'sDSb', 'AutoId'])
    df = df.dropna() 
    # Convert categorical variables to numerical using Label Encoding
    label_encoders = {}
    for column in ['Proto', 'Cause', 'Flgs' ]:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le

    # Split df into features and labels
    X = df.drop(columns=['Attack'])  # Assuming 'label' is the target variable
    y = df['Attack']
    
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        
        # Map unseen values to '<unknown>'
        y_test = y_test.map(lambda s: '<unknown>' if s not in le.classes_ else s)

        # Add '<unknown>' to classes and transform
        le.classes_ = np.append(le.classes_, '<unknown>')
        y_test = le.transform(y_test)
        
        
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')

    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.2821764794785289, 'accMean': 0.9741876465881502, 'recMean': 0.27529942387454687, 'precMean': 0.40047926608942747}
Std of all scores:  {'f1Std': 0.06584206622083554, 'accStd': 0.012850114147820602, 'recStd': 0.07730881656388865, 'precStd': 0.1057355854499046}
_______________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.282176551873622, 'accMean': 0.9742940268839515, 'recMean': 0.2755756362318677, 'precMean': 0.400690457214823}
Std of all scores:  {'f1Std': 0.06438636659744947, 'accStd': 0.012881245839806606, 'recStd': 0.07842694837153792, 'precStd': 0.10099666807049751}
_______________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scor

In [11]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  60
Mean Scores (Best):  {'f1Mean': 0.2958043765731388, 'accMean': 0.9748607770805486, 'recMean': 0.28066591959078196, 'precMean': 0.4332457980772425}
Std Scores (Best): {'f1Std': 0.06929691709657923, 'accStd': 0.012086436958630667, 'recStd': 0.07653075689396195, 'precStd': 0.10235572875915444}


In [12]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  1
Mean Scores (Worst):  {'f1Mean': 0.2692062383148125, 'accMean': 0.9747837430732442, 'recMean': 0.2558095383492639, 'precMean': 0.4028987273454387}
Std Scores (Worst): {'f1Std': 0.053174558252051835, 'accStd': 0.011773501001246127, 'recStd': 0.06524438093709921, 'precStd': 0.10251763584389914}


In [13]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/ET_unsw_argus.json', 'w') as f:
    json.dump(results, f, indent=4)